In [5]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import trange, tqdm

In [20]:
import torch
torch.cuda.is_available()

True

In [7]:
data = pd.read_csv('../Dataset/0OYkPK', sep=",", header=None)
data.columns = data.iloc[0]
data = data[1:]
data['full'] = data['sentence1']+ " " + data['sentence2']+ " " + data['sentence3']+ " " + data['sentence4']+ " " + data['sentence5']
data['input'] = data['sentence1']+ " " + data['sentence2']+ " " + data['sentence3']+ " " + data['sentence4']


In [16]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model = GPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)

next(model.parameters()).is_cuda
device = torch.device('cuda')
#model.to(device)
model = torch.nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
model.to(device)
#model.cuda()

#device_map = {0: [0, 1, 2],
#            1: [3, 4,5],
#            2: [6, 7, 8],
#            3: [9,10,11]}
#model.parallelize(device_map) # Splits the model across several devices

# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')
# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [10]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

device_map = {0: [0, 1, 2],
            1: [3, 4,5],
            2: [6, 7, 8],
            3: [9,10,11]}
model.parallelize(device_map) # Splits the model across several devices


# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')
# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))



NameError: name 'GPT2TokenizerFast' is not defined

In [5]:
for i in tqdm(range(100)):
    # Story is:
    input_ids = tokenizer.encode(data.input.iloc[i], return_tensors='tf')
    
    greedy_output = model.generate(input_ids, max_length=100)
    len_input = len(data.input.iloc[i])
    output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    pred_sent = output[len_input:output.index('.',len_input)+1]
    expected = data.sentence5.iloc[i]
    
    print('case',  i ,':\n' + 100 * '-')
    print('input:')
    print(data.input.iloc[i])
    print("Expected:")
    print(expected)
    print("Output:")
    print(pred_sent)
    print()


  0%|                                                   | 0/100 [00:00<?, ?it/s]


NameError: name 'tokenizer' is not defined